In [ ]:
import pandas as pd
import numpy as np 


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
dataset=pd.read_csv('mushrooms 2.csv')

In [ ]:
X=dataset.drop('class', axis=1)
y=dataset['class']


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
X_train

In [ ]:
dataset=dataset.drop("veil-color", axis=1)
dataset=dataset.drop("gill-attachment", axis=1)
dataset=dataset.drop("veil-type", axis=1)

In [ ]:
X_train=X_train.drop("veil-color", axis=1)
X_train=X_train.drop("gill-attachment", axis=1)
X_train=X_train.drop("veil-type", axis=1)


In [ ]:
X_val=X_val.drop("veil-color", axis=1)
X_val=X_val.drop("gill-attachment", axis=1)
X_val=X_val.drop("veil-type", axis=1)

In [ ]:
threshold = 0.01  # 1% cutoff
total_rows = len(X_train)

for col in X_train.select_dtypes(include=['object', 'category']).columns:
    freq = X_train[col].value_counts(normalize=True)  # relative frequency per category
    rare = freq[freq < threshold]  # categories below threshold
    if len(rare) > 0:
        print(f"\nOutlier categories in '{col}':")
        for cat, prop in rare.items():
            print(f"  {cat}: {prop*100:.2f}% of data")

In [ ]:
for col in X_train.select_dtypes(include=['object', 'category']).columns:
    freq = X_train[col].value_counts(normalize=True)      
    rare_cats = freq[freq < threshold].index           
    if len(rare_cats) > 0:
        X_train[col] = X_train[col].replace(rare_cats, 'Other')  
        print(f"Replaced rare categories in '{col}' with 'Other'")

print("\n All rare categories (below threshold) have been grouped into 'Other'")

In [ ]:
for col in X_train.select_dtypes(include=['object', 'category']).columns:
    freq = X_train[col].value_counts(normalize=True)   
    rare_cats = freq[freq < threshold].index        
    if len(rare_cats) > 0:
        X_train = X_train[~X_train[col].isin(rare_cats)]     
        print(f"Dropped rows with rare categories in '{col}'")

print(f"\nRemaining rows: {len(X_train)} ({len(X_train)/total_rows*100:.2f}% kept)")

In [ ]:
threshold = 0.01  # 1% cutoff
total_rows = len(X_val)

for col in X_val.select_dtypes(include=['object', 'category']).columns:
    freq = X_val[col].value_counts(normalize=True)  # relative frequency per category
    rare = freq[freq < threshold]  # categories below threshold
    if len(rare) > 0:
        print(f"\nOutlier categories in '{col}':")
        for cat, prop in rare.items():
            print(f"  {cat}: {prop*100:.2f}% of data")

In [ ]:
for col in X_val.select_dtypes(include=['object', 'category']).columns:
    freq = X_val[col].value_counts(normalize=True)      
    rare_cats = freq[freq < threshold].index           
    if len(rare_cats) > 0:
        X_val[col] = X_val[col].replace(rare_cats, 'Other')  
        print(f"Replaced rare categories in '{col}' with 'Other'")

print("\n All rare categories (below threshold) have been grouped into 'Other'")

In [ ]:
for col in X_val.select_dtypes(include=['object', 'category']).columns:
    freq = X_val[col].value_counts(normalize=True)   
    rare_cats = freq[freq < threshold].index        
    if len(rare_cats) > 0:
        X_val = X_val[~X_val[col].isin(rare_cats)]     
        print(f"Dropped rows with rare categories in '{col}'")

print(f"\nRemaining rows: {len(X_val)} ({len(X_val)/total_rows*100:.2f}% kept)")

In [ ]:
mode_stalk_root = X_train['stalk-root'].mode()[0]
X_train['stalk-root'] = X_train['stalk-root'].replace('?', mode_stalk_root)
print(X_train['stalk-root'].value_counts())

In [ ]:
mode_stalk_root = X_val['stalk-root'].mode()[0]
X_val['stalk-root'] = X_val['stalk-root'].replace('?', mode_stalk_root)
print(X_val['stalk-root'].value_counts())

In [ ]:
X_train

In [ ]:
X_train['cap-shape'].unique() #one hot
X_train['cap-surface'].unique() #one hot
X_train['cap-color'].unique() #one hot
X_train['bruises'].unique() #label encode
X_train['odor'].unique() #label encode 
#highly correlated with target so i can encode the bad smells together and the other together as two classes 
X_train['gill-spacing'].unique() #one hot
X_train['gill-size'].unique() #one hot
X_train['gill-color'].unique() #drop
X_train['stalk-shape'].unique() #one hot
X_train['stalk-root'].unique() #one hot


Letter
 
Odor Description
Edibility Association
s	spicy	Poisonous
f	foul	Poisonous
y   fishy   Poisonous
n	none	Mostly Edible
l	lilac	Edible
p	pungent	Poisonous
a	almond	Edible
c	creosote	Poisonous

black: k
brown: n
buff: b
chocolate:h 
gray: g
green: r
orange: o
pink: p
purple: u 
red: e
white: w 
yellow: y
 

In [ ]:
from sklearn.compose import ColumnTransformer


one_hot_encoder = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), X_train.columns.tolist())  # Apply to all columns for demonstration
    ],
    remainder='passthrough'  # Keep other columns as is
)

In [ ]:
# Apply one-hot encoding to X_train
X_train_encoded = one_hot_encoder.fit_transform(X_train)
X_train_encoded = pd.DataFrame(
    X_train_encoded, 
    columns=one_hot_encoder.get_feature_names_out()
)

# Apply one-hot encoding to X_val
X_val_encoded = one_hot_encoder.transform(X_val)
X_val_encoded = pd.DataFrame(
    X_val_encoded, 
    columns=one_hot_encoder.get_feature_names_out()
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42, n_estimators=100)

# Train the model on the one-hot encoded training data
rf_classifier.fit(X_train_encoded, y_train)

# Make predictions on the validation set
y_val_pred = rf_classifier.predict(X_val_encoded)

# Evaluate the model
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {accuracy:.2f}")

# Print classification report for detailed metrics
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred))

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
dataset = pd.read_csv('mushrooms 2.csv')

# Separate features and target
X = dataset.drop('class', axis=1)
y = dataset['class']

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Drop unnecessary columns
columns_to_drop = ["veil-color", "gill-attachment", "veil-type"]
X_train = X_train.drop(columns=columns_to_drop, axis=1)
X_val = X_val.drop(columns=columns_to_drop, axis=1)

# Handle rare categories in X_train and X_val
threshold = 0.01  # 1% cutoff

def handle_rare_categories(X, y):
    total_rows = len(X)
    for col in X.select_dtypes(include=['object', 'category']).columns:
        freq = X[col].value_counts(normalize=True)
        rare_cats = freq[freq < threshold].index
        if len(rare_cats) > 0:
            mask = ~X[col].isin(rare_cats)  # Mask for rows to keep
            X = X[mask]
            y = y[mask]  # Drop corresponding rows in y
            print(f"Dropped rows with rare categories in '{col}'")
    print(f"\nRemaining rows: {len(X)} ({len(X)/total_rows*100:.2f}% kept)")
    return X, y

X_train, y_train = handle_rare_categories(X_train, y_train)
X_val, y_val = handle_rare_categories(X_val, y_val)

# Impute missing values in 'stalk-root'
mode_stalk_root_train = X_train['stalk-root'].mode()[0]
X_train['stalk-root'] = X_train['stalk-root'].replace('?', mode_stalk_root_train)

mode_stalk_root_val = X_val['stalk-root'].mode()[0]
X_val['stalk-root'] = X_val['stalk-root'].replace('?', mode_stalk_root_val)

# One-hot encode the categorical features
one_hot_encoder = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), X_train.columns.tolist())
    ],
    remainder='passthrough'
)

# Apply one-hot encoding to X_train
X_train_encoded = one_hot_encoder.fit_transform(X_train)
X_train_encoded = pd.DataFrame(
    X_train_encoded, 
    columns=one_hot_encoder.get_feature_names_out()
)

# Apply one-hot encoding to X_val
X_val_encoded = one_hot_encoder.transform(X_val)
X_val_encoded = pd.DataFrame(
    X_val_encoded, 
    columns=one_hot_encoder.get_feature_names_out()
)

# Train a Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42, n_estimators=100)
rf_classifier.fit(X_train_encoded, y_train)

# Make predictions on the validation set
y_val_pred = rf_classifier.predict(X_val_encoded)

# Evaluate the model
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {accuracy:.2f}")

# Print classification report for detailed metrics
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred))

Dropped rows with rare categories in 'cap-shape'
Dropped rows with rare categories in 'cap-surface'
Dropped rows with rare categories in 'cap-color'
Dropped rows with rare categories in 'odor'
Dropped rows with rare categories in 'gill-color'
Dropped rows with rare categories in 'stalk-surface-above-ring'
Dropped rows with rare categories in 'stalk-color-above-ring'
Dropped rows with rare categories in 'stalk-color-below-ring'
Dropped rows with rare categories in 'ring-type'
Dropped rows with rare categories in 'spore-print-color'

Remaining rows: 6098 (93.83% kept)
Dropped rows with rare categories in 'cap-shape'
Dropped rows with rare categories in 'cap-surface'
Dropped rows with rare categories in 'cap-color'
Dropped rows with rare categories in 'odor'
Dropped rows with rare categories in 'gill-color'
Dropped rows with rare categories in 'stalk-surface-above-ring'
Dropped rows with rare categories in 'stalk-color-above-ring'
Dropped rows with rare categories in 'stalk-color-below-ri